In [1]:
import pandas as pd
import json
from shapely.geometry import shape
import csv

In [3]:
#generate point file: point name aligns with linstring source and target name
f = open('case_ACTIVSg10k point.json')
data = json.load(f)
points = []
for feature in data:
    if feature['geometry']['type'] == 'Point':
        geo = shape(feature["geometry"])
        p = feature["properties"]
        # kvlevels = p["KVlevels"].replace("[", "{")
        # kvlevels = kvlevels.replace("]", "}")
        points.append({
            "wkt":geo.wkt,
            "bus_name": p["NAME"],
            "kilovolt levels": p["KVlevels"],
            "number of buses":p["nbus"],
            "vm":p["Vm"],
            "start":p["start"],
        })



keys = points[0].keys()

with open('10k_bus_7_26.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(points)

In [4]:
# assign id to combined points

# df = pd.read_csv('10k_bus_7_26.csv')
# df['id'] = df.groupby(['wkt']).ngroup()
# df['id'] = df['bus_name'].map(str) + "_" +df['id'].map(str)
# df.to_csv('10k_bus_id_7_26.csv')

In [6]:
# update point id in us file

# df = pd.read_csv('/content/drive/MyDrive/23 summer pnnl/new_all_bus_id_7_25.csv')
# f = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all point.json')
# pointsdata = json.load(f)

# for point in pointsdata:
#     src = str(point['geometry']['coordinates'][0]) + ' '+ str(point['geometry']["coordinates"][1])
#     result = df.loc[df['wkt'] == src, 'id']
#     if(len(result.index)>0):
#         point['properties']['NAME'] = result.iloc[0]
#     # df[df.wkt == src].iloc[0].id
# with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all point new name.json', 'w') as f:
#     json.dump(pointsdata, f)

In [5]:
def getGeneration(data):
    Geni = None
    Gens = []
    minPg = 1000.0
    maxPg = 0.0
    Pgcoal = 0.0
    Pghydro = 0.0
    Pgnuclear = 0.0
    Pgng = 0.0
    Pgsolar = 0.0
    Pgwind = 0.0
    Pgother = 0.0
    Pgcoalcap = 0.0
    Pghydrocap = 0.0
    Pgnuclearcap = 0.0
    Pgngcap = 0.0
    Pgsolarcap = 0.0
    Pgwindcap = 0.0
    Pgothercap = 0.0

    for feature in data:
        if feature['geometry']['type'] == 'Point':
            subst = feature['properties']
            name = subst['NAME']
            nbus = subst['nbus']
            Pg = 0.0
            Pcap = 0.0
            gen_fuel = ''
            ngen = 0
            KV = []
            for bus in subst['bus']:
                KV.append(bus['BASE_KV'])
                
                for gen in bus['gen']:
                    Pg += gen['GEN_STATUS'] * gen['PG']
                    Pcap += gen['GEN_STATUS'] * gen['PMAX']
                    gen_fuel = gen['GEN_FUEL'].lower()

                    if gen_fuel == 'wind':
                        Pgwind += gen['GEN_STATUS'] * gen['PG']
                        Pgwindcap += gen['GEN_STATUS'] * gen['PMAX']
                    elif gen_fuel == 'solar':
                        Pgsolar += gen['GEN_STATUS'] * gen['PG']
                        Pgsolarcap += gen['GEN_STATUS'] * gen['PMAX']
                    elif gen_fuel == 'coal':
                        Pgcoal += gen['GEN_STATUS'] * gen['PG']
                        Pgcoalcap += gen['GEN_STATUS'] * gen['PMAX']
                    elif gen_fuel == 'nuclear':
                        Pgnuclear += gen['GEN_STATUS'] * gen['PG']
                        Pgnuclearcap += gen['GEN_STATUS'] * gen['PMAX']
                    elif gen_fuel == 'hydro':
                        Pghydro += gen['GEN_STATUS'] * gen['PG']
                        Pghydrocap += gen['GEN_STATUS'] * gen['PMAX']
                    elif gen_fuel == 'ng':
                        Pgng += gen['GEN_STATUS'] * gen['PG']
                        Pgngcap += gen['GEN_STATUS'] * gen['PMAX']
                    else:
                        Pgother += gen['GEN_STATUS'] * gen['PG']
                        Pgothercap += gen['GEN_STATUS'] * gen['PMAX']
                    ngen += 1

            if ngen:
                color = ''
                if gen_fuel == 'wind':
                    color = 'green'
                elif gen_fuel == 'solar':
                    color = 'yellow'
                elif gen_fuel == 'coal':
                    color = 'gray'
                elif gen_fuel == 'nuclear':
                    color = 'red'
                elif gen_fuel == 'hydro':
                    color = 'blue'
                elif gen_fuel == 'ng':
                    color = 'orange'
                else:
                    color = 'black'

                if Pg <= minPg:
                    minPg = Pg
                if Pg >= maxPg:
                    maxPg = Pg
                geo = shape(feature["geometry"])
                Geni = { "coordinates": geo.wkt, "Pg": Pg, "Pcap": Pcap, "KVlevels": KV, "color": color, "name":name, "nbus": nbus, "type": gen_fuel }
                Gens.append(Geni)
    return { "minPg": minPg, "maxPg": maxPg, "Gens": Gens, "Pgwind": Pgwind, "Pgsolar": Pgsolar, "Pgnuclear": Pgnuclear, "Pghydro": Pghydro, "Pgng": Pgng, "Pgcoal": Pgcoal, "Pgother": Pgother, "Pgwindcap": Pgwindcap, "Pgsolarcap": Pgsolarcap, "Pgnuclearcap": Pgnuclearcap, "Pghydrocap": Pghydrocap, "Pgngcap": Pgngcap, "Pgcoalcap": Pgcoalcap, "Pgothercap": Pgothercap }



In [7]:

f = open('case_ACTIVSg10k point.json')
data = json.load(f)
gendata = getGeneration(data)

keys = gendata['Gens'][0].keys()

with open('10k_generation_7_26.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(gendata['Gens'])

In [11]:
# update transmission line with source, target and actual flow 
f = open('case_ACTIVSg10k line.json')
data = json.load(f)
lines = []
for feature in data:
    geo = shape(feature["geometry"])
    p = feature["properties"]
    # kvlevels = p["KVlevels"].replace("[", "{")
    # kvlevels = kvlevels.replace("]", "}")
    x = p["NAME"].split(' -- ')
    if(p['PF']>0):
        lines.append({
            "wkt":geo.wkt,
            "flow capacity":p["RATE_A"],
            "pf":p["PF"],
            "qf":p["QF"],
            "pt":p["PT"],
            "qt":p["QT"],
            "kilovolt": p["KV"],
            "line_name": p["NAME"],
            "srouce": x[0],
            "target": x[1],
            "actual flow":abs(p["PF"]),
            

        })
    else:
        lines.append({
            "wkt":geo.wkt,
            "flow capacity":p["RATE_A"],
            "pf":p["PF"],
            "qf":p["QF"],
            "pt":p["PT"],
            "qt":p["QT"],
            "kilovolt": p["KV"],
            "line_name": p["NAME"],
            "srouce": x[1],
            "target": x[0],
            "actual flow":abs(p["PF"]),
        })



keys = lines[0].keys()

with open('10k_transmission_line_7_27.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(lines)

In [10]:
# change line name to uniqe point id
df = pd.read_csv('/content/drive/MyDrive/23 summer pnnl/new_all_bus_id_7_25.csv')
f = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all line.json')
linedata = json.load(f)
# lines = []
# df['temp_wkt'] = df['wkt'].map(str).replace("POINT (", '')
# df['temp_wkt'] = df['temp_wkt'].map(str).replace(')','')
for line in linedata:
    # print(line['geometry']['coordinates'][0])+
    src = str(line['geometry']['coordinates'][0][0]) + ' '+ str(line['geometry']["coordinates"][0][1])
    det = str(line['geometry']["coordinates"][1][0]) + ' '+ str(line['geometry']["coordinates"][1][1])
    if(float(line['properties']['PF'])>0):
        sresult = df.loc[df['wkt'] == src, 'id']
        tresult = df.loc[df['wkt'] == det, 'id']
        if(len(sresult.index)>0 and len(tresult.index)>0):
          line['properties']['source'] = sresult.iloc[0]
          line['properties']['target'] = tresult.iloc[0]
          line['properties']['NAME'] = line['properties']['source'] + ' -- ' + line['properties']['target']
    else:
        sresult = df.loc[df['wkt'] == src, 'id']
        tresult = df.loc[df['wkt'] == det, 'id']
        if(len(sresult.index)>0 and len(tresult.index)>0):
          line['properties']['target'] = sresult.iloc[0]
          line['properties']['source'] = tresult.iloc[0]
          line['properties']['NAME'] = line['properties']['target'] + ' -- ' + line['properties']['source']

with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all line new name.json', 'w') as f:
    json.dump(linedata, f)




In [11]:
linejson = {
      "type": "FeatureCollection",
      "features":linedata
    }
with open('/content/drive/MyDrive/23 summer pnnl/qgis all line new name.json', 'w') as f:
    json.dump(linejson, f)

In [12]:

f = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all temp.json')
alldata = json.load(f)
alldata['geojsondata']['features'] = linedata + pointsdata
alldata['nbus'] = len(pointsdata)
alldata['nbranch'] = len(linedata)
with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all.json', 'w') as f:
    json.dump(alldata, f, indent=4)

In [ ]:
# keys = lines[0].keys()

# with open('/content/drive/MyDrive/23 summer pnnl/all_line_id_7_24.csv', 'w', newline='') as output_file:
#     dict_writer = csv.DictWriter(output_file, keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(lines)

In [25]:
# combine 2000 dataset with other dataset
f70k = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg70k list.json')
d70k = json.load(f70k)

f10k = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg10k list.json')
d10k = json.load(f10k)
f2000 = open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg2000 list.json')
d2000 = json.load(f2000)


p70 = [d for d in d70k if d['geometry']['type'] == 'Point']
p10 = [d for d in d10k if d['geometry']['type'] == 'Point']
p2000 = [d for d in d2000 if d['geometry']['type'] == 'Point']
points = p70 + p10
points = points + p2000
with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all point.json', 'w') as f:
    json.dump(points, f)

l70 = [d for d in d70k if d['geometry']['type'] == 'LineString']
l10 = [d for d in d10k if d['geometry']['type'] == 'LineString']
l2000 = [d for d in d2000 if d['geometry']['type'] == 'LineString']
lines = l70 + l10
lines = lines + l2000
with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all line.json', 'w') as f:
    json.dump(lines, f)

In [26]:
with open('/content/drive/MyDrive/23 summer pnnl/case_ACTIVSg all point.json', 'w') as f:
    json.dump(points, f)

In [ ]:
#generate line csv


In [15]:
# transmission line process
# generate source and target info of transmission lines, make pf value absolute
#remove linestring where the end and start point is the same, because it's not valid to perform bigquery
import csv
input = open('/content/drive/MyDrive/23 summer pnnl/all_line_part_attribute_db_7_25.csv', 'rt')
output = open('/content/drive/MyDrive/23 summer pnnl/all_line_db_7_26.csv', 'w', newline='')
writer = csv.writer(output)
reader = csv.reader(input)
headers = next(reader, None)  # returns the headers or `None` if the input is empty
if headers:
    headers.append('actual flow')
    writer.writerow(headers)
for row in reader:
    row.append(abs(float(row[2])))
    writer.writerow(row)
input.close()
output.close()

In [ ]:
# generate text from point table
# Geni = { "coordinates": geo.wkt, "Pg": Pg, "Pcap": Pcap, "KVlevels": KV, "color": color, "name":name, "nbus": nbus, "type": gen_fuel }
with open('200point7_11.txt', 'a') as the_point_file:
        the_point_file.write('This file lists a number of power grid generations and their information, including individual id, name, location cooridnates in WKT format (i.e., latitude and longitude), power produced, power capacity, voltage level, number of buses and category (e.g., wind, nuclear). Each line represents one generation.\n')
        for gen in gendata['Gens']:
            strline = ( "I am a " + gen['type'] +" generation. My name is " + gen['name'] +
                       ". My location coordinates is defined by " + gen["coordinates"] +
                       ". The amount of power generated by me is " + str(gen['Pg']) +
                       ". My power capacity is " + str(gen['Pcap']) +
                       ". My voltage level range is " + str(gen['KVlevels']) +
                       ". I have " + str(gen['nbus']) + " buses.\n"
                        )
            the_point_file.write(strline)


In [ ]:
#generate text from line table
with open('200line7_11noloop.csv') as f:
    linedata = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]


with open('200line7_11.txt', 'a') as the_line_file:
    the_line_file.write('This file lists a number of power grid transmission lines and their information, including individual name, location cooridnates in WKT format (i.e., latitude and longitude of the source and target point), flow capacity, voltage level, actual flow. Each line represents one transimisson line. ' +
                        'The trnasmission line name is defined by join its source and target node name with \' -- \'. '
                        + 'If the PF value is positive, the first part of its name is the name of the source node and the second part of its name is the name of the target node ' +
                        'If the PF value is negative, the first part of its name is the name of the target node and the second part of its name is the name of the source node\n')
    for line in linedata:
        strline = ( "I am a transmission line. My name is " + line['NAME'] +
                    ". My location coordinates is defined by " + line["WKT"] +
                    ". My flow capacity is " + str(line['RATE_A']) +
                    ". My voltage level is " + str(line['KV']) +
                    ". The actual flow pass through me is " + str(line['PF']) + ".\n"
                    )
        the_line_file.write(strline)

In [ ]:
#remove linestring where the end and start point is the same, because it's not valid to perform bigquery
import csv
input = open('200line7_11.csv', 'rt')
output = open('200line7_11noloop.csv', 'w', newline='')
writer = csv.writer(output)
reader = csv.reader(input)
headers = next(reader, None)  # returns the headers or `None` if the input is empty
if headers:
    writer.writerow(headers)
for row in reader:
    x = row[1].split(",")
    if x[0]!=x[1]:
        row[1] = "LINESTRING (" + row[1] + ")"
        writer.writerow(row)
input.close()
output.close()

In [7]:
# add statename to county table 
import pandas as pd
import us

county_df = pd.read_csv('counties.csv')

fips_to_name = us.states.mapping("fips", "name")

county_df["fips"] = county_df["fips"].map(str).str.zfill(2)
county_df["states"] = county_df["fips"].map(fips_to_name)

county_df.to_csv('county_name.csv')